# Plot results
Plot the gridded fields and the associated error fields.

In [41]:
using Pkg
Pkg.activate(".")
Pkg.add(Pkg.PackageSpec(; name = "GeoMakie", version = "0.7.12"))
using Glob
using Dates
using ColorSchemes
using NCDatasets
using CairoMakie, GeoMakie
using GeoDatasets
using GeometryOps, GeoInterface
include("./config.jl")
using JupyterFormatter
enable_autoformat()

  Activating project at `~/Projects/EMODnet-Chemistry-GriddedMaps/src`
   Resolving package versions...
  No Changes to `~/Projects/EMODnet-Chemistry-GriddedMaps/src/Project.toml`
  No Changes to `~/Projects/EMODnet-Chemistry-GriddedMaps/src/Manifest.toml`


2-element Vector{Function}:
 format_current_cell (generic function with 1 method)
 format_current_cell (generic function with 1 method)

### Select variable, month and vertical level (depth)
- Water_body_ammonium
- Water_body_chlorophyll-a
- Water_body_dissolved_inorganic_nitrogen
- Water_body_dissolved_oxygen_concentration
- Water_body_phosphate
- Water_body_silicate

In [47]:
varname = "Water_body_dissolved_oxygen_concentration"
varname = "Water_body_phosphate"
varname_ = replace(varname, "_" => " ")
depth2plot = 25
month2plot = 5
cmap = cgrad(:RdYlBu, rev = true)
cmaperror = cgrad(:RdYlGn_10, 10, rev = true, categorical = true)

## Files and directories

In [3]:
databasedir = "/home/ctroupin/data/EMODnet-Chemistry/Eutrophication2024/Results/"
databasedir = "/home/ctroupin/data/EMODnet/EMODnet-Chemistry/2024/"
datadir = joinpath(databasedir, "../netCDF")
casename = "$(varname)-res-0.25-epsilon2-2.0-varlen1-lb5-maxit-5000-reltol-1.0e-9-bathcl-go-exclude-mL-1960-exNS2-monthly"
resultfile = joinpath(databasedir, casename, "Results/$(varname)_monthly.nc")
isfile(resultfile)# , isdir(datadir)

false

## Read fields
### DIVAnd

In [4]:
lon, lat, depth, dates, field, error =
    read_results(resultfile, varname, depth2plot, month2plot);

LoadError: NetCDF error: [31mOpening path /home/ctroupin/data/EMODnet/EMODnet-Chemistry/2024/Water_body_phosphate-res-0.25-epsilon2-2.0-varlen1-lb5-maxit-5000-reltol-1.0e-9-bathcl-go-exclude-mL-1960-exNS2-monthly/Results/Water_body_phosphate_monthly.nc: No such file or directory[39m (NetCDF error code: 2)

### WOA fields

In [ ]:
if varname == "Water_body_dissolved_oxygen_concentration"
    woavarname = "o"
elseif varname == "Water_body_phosphate"
    woavarname = "p"
end

In [ ]:
woafile = joinpath(woadir, "woa23_all_$(woavarname)$(lpad(month2plot, 2, '0'))_01.nc")
@info("Getting data from $(woafile)")
if isfile(woafile)
    lonwoa, latwoa, depthwoa, dateswoa, fieldwoa, errorwoa =
        read_woa(woafile, varname, depth2plot, lonr, latr)
else
    @warn("$(woafile) does not exist")
end

## Make the plots
### DIVAnd

In [ ]:
f1, ga1, hm1 = plot_field_var(
    varname,
    lon,
    lat,
    field,
    depth2plot,
    month2plot,
    "DIVAnd",
    cmap,
    vmin = 225.0,
    vmax = 375.0,
)
save(joinpath(figdir, "$(varname)_depth-$(depth2plot)_month-$(month2plot)_DIVAnd.png"), f1)

### WOA

In [ ]:
f2, ga2, hm2 = plot_field_var(
    varname,
    lonwoa,
    latwoa,
    fieldwoa,
    depth2plot,
    month2plot,
    "WOA",
    cmap,
    vmin = 225.0,
    vmax = 375.0,
)
save(joinpath(figdir, "$(varname)_depth-$(depth2plot)_month-$(month2plot)_WOA.png"), f2)

## Error fields
### DIVAnd

In [ ]:
include("./config.jl")
f1, ga1, hm1 = plot_error_field(varname, lon, lat, error, depth2plot, month2plot, "DIVAnd")
display(f1)

In [ ]:
f2, ga2, hm2 =
    plot_error_field(varname, lonwoa, latwoa, errorwoa * 100, depth2plot, month2plot, "WOA")
display(f2)

## Plot all the annex fields
### Read measurements

In [5]:
resultfile = "/home/ctroupin/data/EMODnet/EMODnet-Chemistry/2024/Results/Water_body_phosphate.nc"
isfile(resultfile)

true

In [6]:
function read_annex_fields(resultfile::String)

    NCDataset(resultfile, "r") do ds
        # print(ds)
        lon = ds["lon"][:]
        lat = ds["lat"][:]
        depth = ds["depth"][:]
        dates = ds["time"][:]

        # Subsetting
        depthindex = findfirst(depth .== depth2plot)
        timeindex = findfirst(Dates.month.(dates) .== month2plot)
        # Read gridded field and error
        field = coalesce.(ds[varname][:, :, depthindex, timeindex], NaN)
        fielderror = coalesce.(ds[varname*"_relerr"][:, :, depthindex, timeindex], NaN)
        fieldL1 = coalesce.(ds[varname*"_L1"][:, :, depthindex, timeindex], NaN)
        fieldL2 = coalesce.(ds[varname*"_L2"][:, :, depthindex, timeindex], NaN)
        fielddeepest = coalesce.(ds[varname*"_deepest"][:, :, timeindex], NaN)

        return lon::Vector{Float64},
        lat::Vector{Float64},
        depth::Vector{Float64},
        dates::Vector{DateTime},
        field::Matrix{AbstractFloat},
        fielderror::Matrix{AbstractFloat},
        fieldL1::Matrix{AbstractFloat},
        fieldL2::Matrix{AbstractFloat},
        fielddeepest::Matrix{AbstractFloat}
    end
end

read_annex_fields (generic function with 1 method)

In [7]:
lon, lat, depth, dates, field, fielderror, fieldL1, fieldL2, fielddeepest =
    read_annex_fields(resultfile);

### Plot all the fields together

In [ ]:
include("./config.jl")
f3 = plot_additional_field(
    varname,
    varunits[varname],
    lon,
    lat,
    fielddeepest,
    fieldL1,
    fieldL2,
    fielderror,
    depth2plot,
    month2plot,
)
save(
    joinpath(
        figdir,
        "$(varname)_depth-$(depth2plot)_month-$(month2plot)_additional_fields.png",
    ),
    f3,
)

### Plot all the fields one-by-one

In [60]:
include("./config.jl")

plot_field_var_fast (generic function with 3 methods)

In [51]:
f1, ga1, hm1 = plot_field_var(
    varname,
    lon,
    lat,
    field,
    depth2plot,
    month2plot,
    "DIVAnd",
    cmap,
    vmin = first(varrange[varname]),
    vmax = last(varrange[varname]) / 5.0,
)
save(
    joinpath(figdir, "$(varname)_depth-$(depth2plot)_month-$(month2plot)_DIVAnd_field.png"),
    f1,
)

In [ ]:
f1, ga1, hm1 = plot_field_var(
    varname,
    lon,
    lat,
    fieldL1,
    depth2plot,
    month2plot,
    "DIVAnd",
    cmap,
    vmin = first(varrange[varname]),
    vmax = last(varrange[varname]) / 5.0,
)
save(
    joinpath(
        figdir,
        "$(varname)_depth-$(depth2plot)_month-$(month2plot)_DIVAnd_fieldL1.png",
    ),
    f1,
)

In [53]:
f1, ga1, hm1 = plot_field_var(
    varname,
    lon,
    lat,
    fieldL2,
    depth2plot,
    month2plot,
    "DIVAnd",
    cmap,
    vmin = first(varrange[varname]),
    vmax = last(varrange[varname]) / 5.0,
)
save(
    joinpath(
        figdir,
        "$(varname)_depth-$(depth2plot)_month-$(month2plot)_DIVAnd_fieldL2.png",
    ),
    f1,
)

In [61]:
f1, ga1, hm1 = plot_field_var_deepest(
    varname,
    lon,
    lat,
    fielddeepest,
    month2plot,
    "DIVAnd",
    cmap,
    vmin = first(varrange[varname]),
    vmax = last(varrange[varname]),
)
save(
    joinpath(
        figdir,
        "$(varname)_depth-$(depth2plot)_month-$(month2plot)_DIVAnd_fielddeepest.png",
    ),
    f1,
)

In [58]:
f1, ga1, hm1 =
    plot_error_field(varname, lon, lat, fielderror, depth2plot, month2plot, "DIVAnd")
save(
    joinpath(
        figdir,
        "$(varname)_depth-$(depth2plot)_month-$(month2plot)_DIVAnd_fielderror.png",
    ),
    f1,
)